# Introduction

AIM : create a self learning model, that has the ability to detect the a specific object on which the model is trained on field.


STEPS :

step 1 : placing the tag just above the object and take photos.(almost 100-120 photos)

step 2 : perform tag detection on it and save the images on which the tag is detected in different folder.

step 3 : create a bounding box just below the tag and generate the txt file for the object (according to the size of object).

step 4 : augment the images along with the txt files.

step 5 : train the model for object.

# Imports



In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.3/107.7 GB disk)


In [ ]:
!pip install -U ultralytics

In [ ]:
!ultralytics --version

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ argument '--version' does not require leading dashes '--', updating to 'version'.
8.2.2


In [ ]:
from ultralytics import YOLO

# Drive Mount

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Set Path

change directory to the folder in which we want to save the predict folder for tag detection on dataset.

In [ ]:
%cd /content/drive/MyDrive/Last/Tag

/content/drive/MyDrive/Last/Tag


# tag detection on dataset

predicting the tag

model - the model for tag detection

source - dataset (100 clicked images)

In [ ]:
!yolo task=detect mode=predict model="/content/drive/MyDrive/self-learn/runs/detect/train10/weights/best.pt" conf=0.25 source="/content/drive/MyDrive/Sharpener" save=true plots=true

2024-04-10 09:23:29.425723: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 09:23:29.425857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 09:23:29.428466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 09:23:31.719129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
image 1/34 /content/drive/MyDrive/Sharpener/IMG_20240402_141636.jpg: 640x480 1 tag, 699.7ms
image

# extract imformation from result


In [ ]:
# load the model
model = YOLO( "/content/drive/Shareddrives/x/tag_detect/train10/weights/best.pt")

Knowing the size of images

In [ ]:
import os
import torch
import cv2
from PIL import Image


# Path to the folder containing images
folder_path = "/content/drive/MyDrive/Sharpener"

# List all files in the folder
files = os.listdir(folder_path)

# Filter out only the image files (you may need to adjust this depending on the types of images you have)
image_files = [f for f in files if f.endswith((".jpg", ".jpeg", ".png", ".gif"))]

for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_file)

    image = Image.open(image_path)

    # Get the size of the image
    width, height = image.size

    print(f"Image width: {width}, Image height: {height}")
    break

Image width: 3000, Image height: 4000


**Extracting results and saving it to an list named "n"**

**Masking of tag - reducing overfitting induced due to presence of tag in every image**

In [ ]:
import os
import torch
import cv2
import numpy as np
import shutil
import random

# ------------------------------------------------------
# for masking tags in images
def mask_objects(image, bboxes):
    mask = np.zeros_like(image)
    for bbox in bboxes:
        x, y, w, h = bbox
        bbox_mask = np.zeros_like(image)
        bbox_mask[0:y+int(h/2), 0:width] = 1
        mask = np.maximum(mask, bbox_mask)
    masked_image = np.where(mask == 1, (128, 128, 128), image)

    # Save the masked image
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, masked_image)
# -------------------------------------------------------


# Path to the folder containing images
folder_path = "/content/drive/MyDrive/Sharpener"
output_folder = "/content/drive/MyDrive/final/code_modify"

# List all files in the folder
files = os.listdir(folder_path)

# Filter out only the image files (you may need to adjust this depending on the types of images you have)
image_files = [f for f in files if f.endswith((".jpg", ".jpeg", ".png", ".gif"))]


# --------------------------------------
# randomly suffles the images, so that the random images are going to masked
random.shuffle(image_files)   # Shuffle the images
num_images = len(image_files)  # number of images on which the tag is detected
# ---------------------------------------------------


# Define a list to store all tensor values
n = []
cou = 0
# Iterate over each image file
for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_file)

    # Apply your model to the image
    results = model(image_path)

    # Iterate over each result for the image
    for result in results:
        # Extract tensor values and append to the list
        bbox = result.boxes.xywh.tolist()
        bbox = [[int(element) for element in sublist] for sublist in bbox]
        predict_class = result.boxes.cls.tolist()
        image_name = os.path.splitext(image_file)[0]

        # if the image contains tag and it is detected
        if(len(bbox) > 0):
            # saving the image(in which tag is detected) in separate folder
            img = cv2.imread(image_path)
            filename = os.path.basename(image_path)
            # cv2.imwrite(save_path, img)

            destination_path = os.path.join(output_folder, filename)
            if(cou < num_images/2):
              mask_objects(img, bbox)
            else:
              shutil.copyfile(image_path, destination_path)

            # saving the results in a list named "n"
            n.append({"image_name": image_name, "bbox": bbox, "predict_class": predict_class})
            cou += 1


# Print all stored tensor values
# for tensor_value in n:
#     print(tensor_value)
#     cou = cou + 1


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


# txt file generation
The txt file annotation format is supported by yolov8

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

folder_path = save_folder = "/content/drive/MyDrive/Last/data"

# function for creating bounding box just below the image
def create_bounding_box(tag_bbox):

    x, y, w, h = tag_bbox # x and y are coordinates of center
    new_x = x
    new_y = y + int(h/2) + 250
    new_w = 500    # i made 200x200 bounding box just below the tag, increased size to 400x400
    new_h = 500

    # print(x, y, w, h)
    return (new_x, new_y, new_w, new_h)

# function for normalilzing the coordinates for yolov8
def normal(new_bbox):
    x, y, w, h = new_bbox
    n_x = x/width
    n_y = y/height
    n_w = w/width
    n_h = h/height
    return (n_x, n_y, n_w, n_h)


for j in range(len(n)):


    tag_bbox = n[j]["bbox"]  # array of bounding box in the image
    Image_nam = n[j]['image_name']  # name of the image
    filenam = Image_nam + ".txt"    # name of txt file

    filenam = os.path.join(folder_path, filenam)

    # create a empty txt file for a particular image with name -> filenam = Image_nam + ".txt"
    with open(filenam, 'w') as file:
        # print(filenam + "/n")
        pass

    print(len(tag_bbox))
    for i in range(len(tag_bbox)):
        # print(i)
        old_bbox = tag_bbox[i]
        new_bbox = create_bounding_box(old_bbox)   # creating new bounding box below the tag

        new_x, new_y, new_w, new_h = new_bbox

        #  condition if the bounding box is going beyond the edges # 4000 and 3000 is the width and height of the images(in my data set)
        # it is hard coded as of now, we can write code for it in future
        if new_x < 0 or new_x + 500 > width or new_y < 0 or new_y + 500 > height :
           continue
        else :
          n_x, n_y, n_w, n_h = normal(new_bbox)   # normalized values of bbox

          with open(filenam, 'a') as file:
              file.write(f"0 {n_x} {n_y} {n_w} {n_h} \n")  # appending in txt files


/content/drive/MyDrive/Last/data
/content/drive/MyDrive/Last/data/IMG_20240402_142833.txt/n
1
0
1545 2001 643 500
/content/drive/MyDrive/Last/data/IMG_20240402_142228.txt/n
1
0
1573 1893 720 596
/content/drive/MyDrive/Last/data/IMG_20240402_143343.txt/n
1
0
669 2052 647 568
/content/drive/MyDrive/Last/data/IMG_20240402_142249.txt/n
1
0
1459 1971 629 567
/content/drive/MyDrive/Last/data/IMG_20240402_142012.txt/n
1
0
1548 2186 713 730
/content/drive/MyDrive/Last/data/IMG_20240402_141643.txt/n
1
0
1744 2412 863 761
/content/drive/MyDrive/Last/data/IMG_20240402_141830.txt/n
1
0
1593 1421 840 787
/content/drive/MyDrive/Last/data/IMG_20240402_142852__01.txt/n
1
0
1609 1340 760 620
/content/drive/MyDrive/Last/data/IMG_20240402_142624.txt/n
1
0
1510 2031 728 655
/content/drive/MyDrive/Last/data/IMG_20240402_143323.txt/n
1
0
1504 1920 724 647
/content/drive/MyDrive/Last/data/IMG_20240402_142852.txt/n
1
0
1562 1552 776 622
/content/drive/MyDrive/Last/data/IMG_20240402_142247.txt/n
1
0
1354 2184 

# main Augment - run krna hai ise

Albumentation : Albumentations is a powerful open-source image augmentation library.
using Albumentation library for different augmentation :
1. Vertical flip
2. Defocus
3. Gray
4. Scale
5. Rotate
7. Blur


https://albumentations.ai/docs/getting_started/transforms_and_targets/


The augmentation was needed to be done coz too much similarity in the image was overfitting the model, also we have less number of images in over dataset.

In [ ]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

In [ ]:
import shutil
import os

# Source and destination paths
source_folder = '/content/drive/MyDrive/Last/data'
destination_folder = '/content/drive/MyDrive/Last/data2'

# List all files in the source folder
files = os.listdir(source_folder)

# Copy each file to the destination folder
for file in files:
    source_path = os.path.join(source_folder, file)
    destination_path = os.path.join(destination_folder, file)
    shutil.copy(source_path, destination_path)

Vertical

In [ ]:
import cv2
import os


def process_images_in_folder(folder_path, output_directory):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            base_image_path = os.path.join(folder_path, filename)
            base_txt_path = os.path.splitext(base_image_path)[0] + ".txt"
            augment_image_and_points(base_image_path, base_txt_path, output_directory)


def augment_image_and_points(image_path, txt_path, output_dir):
    image = cv2.imread(image_path)
    global img_width, img_height
    img_height, img_width, _ = image.shape
    annotations = read_points_from_txt(txt_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)

    # Scale(annotations, image, 0.5, output_dir, filename)
    Vertical(annotations, image, 0.7, output_dir, filename)


def read_points_from_txt(txt_file):

    # read lines from text file
    with open(txt_file, 'r') as f:
      lines = f.readlines()

    annotations = []

    for line in lines:
        # read data from each line
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        class_id = int(class_id)
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates in coco format/ openCV format
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        box = x1, y1, width, height
        annotations.append((class_id, box))
        return annotations

def Vertical(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.VerticalFlip(p=0.5)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "vertical_flip "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    # plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"vertical_flip {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")

folder_path = "/content/drive/MyDrive/Last/data"
output_directory = "/content/drive/MyDrive/Last/data2"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_images_in_folder(folder_path, output_directory)


Defocus


In [ ]:
import cv2
import os


def process_images_in_folder(folder_path, output_directory):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            base_image_path = os.path.join(folder_path, filename)
            base_txt_path = os.path.splitext(base_image_path)[0] + ".txt"
            augment_image_and_points(base_image_path, base_txt_path, output_directory)


def augment_image_and_points(image_path, txt_path, output_dir):
    image = cv2.imread(image_path)
    global img_width, img_height
    img_height, img_width, _ = image.shape
    annotations = read_points_from_txt(txt_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)

    # Scale(annotations, image, 0.5, output_dir, filename)
    Vertical(annotations, image, 0.7, output_dir, filename)


def read_points_from_txt(txt_file):

    # read lines from text file
    with open(txt_file, 'r') as f:
      lines = f.readlines()

    annotations = []

    for line in lines:
        # read data from each line
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        class_id = int(class_id)
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates in coco format/ openCV format
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        box = x1, y1, width, height
        annotations.append((class_id, box))
        return annotations

def Vertical(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.Defocus(radius=(3, 10), alias_blur=(0.1, 0.5), always_apply=True, p=0.5)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "Defocus "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    # plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"Defocus {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")

folder_path = "/content/drive/MyDrive/Last/data"
output_directory = "/content/drive/MyDrive/Last/data2"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_images_in_folder(folder_path, output_directory)


gray

In [ ]:
import os
import cv2
import shutil

# Path to the folder containing images and text files
folder_path = '/content/drive/MyDrive/Last/data'

# Create a new folder to save the grayscale images
output_folder = '/content/drive/MyDrive/Last/data2'
os.makedirs(output_folder, exist_ok=True)

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.jpg'):  # Assuming images are in JPG format
        # Load the image
        image_path = os.path.join(folder_path, file_name)
        image = cv2.imread(image_path)
        if image is not None:
            # Convert the image to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Save the grayscale image
            new_image_name = file_name.split('.')[0] + '_gray.jpg'
            new_image_path = os.path.join(output_folder, new_image_name)
            cv2.imwrite(new_image_path, gray_image)

            # Copy and rename the corresponding text file
            txt_file_name = file_name.split('.')[0] + '.txt'
            txt_file_path = os.path.join(folder_path, txt_file_name)
            new_txt_file_path = os.path.join(output_folder, new_image_name.split('.')[0] + '.txt')
            shutil.copy(txt_file_path, new_txt_file_path)


Scale

In [ ]:
import cv2
import os


def process_images_in_folder(folder_path, output_directory):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            base_image_path = os.path.join(folder_path, filename)
            base_txt_path = os.path.splitext(base_image_path)[0] + ".txt"
            augment_image_and_points(base_image_path, base_txt_path, output_directory)


def augment_image_and_points(image_path, txt_path, output_dir):
    image = cv2.imread(image_path)
    global img_width, img_height
    img_height, img_width, _ = image.shape
    annotations = read_points_from_txt(txt_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)

    # Scale(annotations, image, 0.5, output_dir, filename)
    Scale(annotations, image, 0.2, output_dir, filename)
    # Vertical(annotations, image, 0.7, output_dir, filename)


def read_points_from_txt(txt_file):

    # read lines from text file
    with open(txt_file, 'r') as f:
      lines = f.readlines()

    annotations = []

    for line in lines:
        # read data from each line
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        class_id = int(class_id)
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates in coco format/ openCV format
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        box = x1, y1, width, height
        annotations.append((class_id, box))
        return annotations

def Scale(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.RandomSizedBBoxSafeCrop(width=img_width, height=img_height, erosion_rate=rate)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "Scale "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    # plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"Scale {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")

folder_path = "/content/drive/MyDrive/Last/data"
output_directory = "/content/drive/MyDrive/Last/data2"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_images_in_folder(folder_path, output_directory)


**Rotate**

In [ ]:
import cv2
import os

# function for the
def process_images_in_folder(folder_path, output_directory):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            base_image_path = os.path.join(folder_path, filename)
            base_txt_path = os.path.splitext(base_image_path)[0] + ".txt"
            augment_image_and_points(base_image_path, base_txt_path, output_directory)


def augment_image_and_points(image_path, txt_path, output_dir):
    image = cv2.imread(image_path)
    global img_width, img_height
    img_height, img_width, _ = image.shape
    annotations = read_points_from_txt(txt_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)

    # Scale(annotations, image, 0.5, output_dir, filename)
    SafeRotate(annotations, image, 0.2, output_dir, filename)
    # Vertical(annotations, image, 0.7, output_dir, filename)


def read_points_from_txt(txt_file):

    # read lines from text file
    with open(txt_file, 'r') as f:
      lines = f.readlines()

    annotations = []

    for line in lines:
        # read data from each line
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        class_id = int(class_id)
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates in coco format/ openCV format
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        box = x1, y1, width, height
        annotations.append((class_id, box))
        return annotations

def SafeRotate(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.SafeRotate(limit=90, p=0.5)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "Rotate "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    # plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"Rotate {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")


# start of code
folder_path = "/content/drive/MyDrive/Last/data"
output_directory = "/content/drive/MyDrive/Last/data2"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_images_in_folder(folder_path, output_directory)


Blur

In [ ]:
import cv2
import os


def process_images_in_folder(folder_path, output_directory):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            base_image_path = os.path.join(folder_path, filename)
            base_txt_path = os.path.splitext(base_image_path)[0] + ".txt"
            augment_image_and_points(base_image_path, base_txt_path, output_directory)


def augment_image_and_points(image_path, txt_path, output_dir):
    image = cv2.imread(image_path)
    global img_width, img_height
    img_height, img_width, _ = image.shape
    annotations = read_points_from_txt(txt_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)

    # Scale(annotations, image, 0.5, output_dir, filename)
    SafeRotate(annotations, image, 0.2, output_dir, filename)
    # Vertical(annotations, image, 0.7, output_dir, filename)


def read_points_from_txt(txt_file):

    # read lines from text file
    with open(txt_file, 'r') as f:
      lines = f.readlines()

    annotations = []

    for line in lines:
        # read data from each line
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        class_id = int(class_id)
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates in coco format/ openCV format
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        box = x1, y1, width, height
        annotations.append((class_id, box))
        return annotations

def SafeRotate(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        # [A.ToGray(p=0.5)],
        [A.Blur(blur_limit=(3, 7), always_apply=True, p=0.5)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "BlurA "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    # plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"BlurA {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")

folder_path = "/content/drive/MyDrive/Last/data"
output_directory = "/content/drive/MyDrive/Last/data2"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_images_in_folder(folder_path, output_directory)


# (no need to run) full augment in one go, but RAM insuffiecient
In colab, we have RAM usage limit, coz of that we were unable to perform all augmentation in a single cell

In [ ]:
import cv2
import os


def process_images_in_folder(folder_path, output_directory):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            base_image_path = os.path.join(folder_path, filename)
            base_txt_path = os.path.splitext(base_image_path)[0] + ".txt"
            augment_image_and_points(base_image_path, base_txt_path, output_directory)


def augment_image_and_points(image_path, txt_path, output_dir):
    image = cv2.imread(image_path)
    global img_width, img_height
    img_height, img_width, _ = image.shape
    annotations = read_points_from_txt(txt_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)


    # Scale(annotations, image, 1, output_dir, filename)
    # Scale(annotations, image, 1.25, output_dir, filename)
    Scale(annotations, image, 0.5, output_dir, filename)
    # Scale(annotations, image, 0.8, output_dir, filename)
    Vertical(annotations, image, 0.7, output_dir, filename)
    # Horizontal(annotations, image, 0.7, output_dir, filename)
    # operations = [
    #     ("original", image, annotations),
    #     ("flip_horizontal", cv2.flip(image, 1), [(cls_id, horizontal(points, image_width)) for cls_id, points in annotations]),
    #     ("flip_vertical", cv2.flip(image, 0), [(cls_id, vertical(points, image_height)) for cls_id, points in annotations])
    # ]

    # for operation, img, ann in operations:
    #     output_image_path = os.path.join(output_dir, f"{name}_{operation}.jpg")
    #     output_txt_path = os.path.join(output_dir, f"{name}_{operation}.txt")
    #     cv2.imwrite(output_image_path, img)
    #     write_points_to_txt(output_txt_path, ann, image_width, image_height)


def write_points_to_txt(txt_file, annotations, image_width, image_height):
    with open(txt_file, 'w') as f:
        for cls_id, box in annotations:
            x, y, w, h = box
            cx =  (x + w/2) / image_width
            cy = (y + h/2) / image_height
            w = w / image_width
            h = h / image_height
            f.write(f"{cls_id} {cx} {cy} {w} {h}\n")

def read_points_from_txt(txt_file):

    # read lines from text file
    with open(txt_file, 'r') as f:
      lines = f.readlines()

    annotations = []

    for line in lines:
        # read data from each line
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        class_id = int(class_id)
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates in coco format/ openCV format
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        box = x1, y1, width, height
        annotations.append((class_id, box))
        return annotations


def Horizontal(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.HorizontalFlip(p=0.5)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "horizontal_flip "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"horizontal_flip {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")



def Scale(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.RandomSizedBBoxSafeCrop(width=img_width, height=img_height, erosion_rate=rate)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "Scale "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"Scale {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")



def Vertical(annotations, image, rate, output_dir, filename):
    bboxes = []
    category_ids = []
    for class_id, box in annotations :
      b = []
      x, y, w, h = box
      b.append(x)
      b.append(y)
      b.append(w)
      b.append(h)
      bboxes.append(b)
      category_ids.append(class_id)


    # # We will use the mapping from category_id to the class name
    # # to visualize the class label for the bounding box on the image
    category_id_to_name = {0: 'sharpener'}

    # augmentation - vertical flip
    transform = A.Compose(
        [A.VerticalFlip(p=0.5)],
        bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
    )
    random.seed(7)
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # saving image
    img = transformed['image'].copy()
    filename_v = "vertical_flip "+ str(rate) + filename
    output_image_path = os.path.join(output_dir, filename_v)
    plt.imshow(img)
    cv2.imwrite(output_image_path, img)


    # creating text file
    name, ext = os.path.splitext(filename)
    output_txt_path = os.path.join(output_dir, f"vertical_flip {rate}{name}.txt")

    with open(output_txt_path, 'w') as f:
      for j in range(len(transformed['bboxes'])):
            x, y, w, h =   transformed['bboxes'][j]
            cx =  (x + w/2) / img_width
            cy = (y + h/2) / img_height
            w = w / img_width
            h = h / img_height
            class_id = transformed["category_ids"][j]
            f.write(f"{class_id} {cx} {cy} {w} {h}\n")


# hello = horizontal(box, image, img_height, img_width, 2)
# hello = horizontal(box, image, img_height, img_width, 0.2)
# hello = horizontal(box, image, img_height, img_width, 0.7)
# hello = horizontal(box, image, img_height, img_width, 1)
# hello = horizontal(box, image, img_height, img_width, 1.5)
# print(hello)

folder_path = "/content/drive/MyDrive/final/object"
output_directory = "/content/drive/MyDrive/final/augment"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_images_in_folder(folder_path, output_directory)





# *Visualization - No need to run it*

for visualization, I used openCV

fistly, I converted the normalized coordinated from txt to the coordinate form of openCV and then plotted a rectangle around object.

In [ ]:
import cv2
import os

# Path to the folder containing images and YOLOv8 txt files
folder_path = '/content/drive/MyDrive/Last/gray'

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):  # Assuming images are JPEG format
        # Load the image
        img_path = os.path.join(folder_path, filename)
        image = cv2.imread(img_path)

        # Load the corresponding YOLOv8 txt file
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        txt_path = os.path.join(folder_path, txt_filename)

        # Read the bounding box coordinates from the YOLOv8 txt file
        with open(txt_path, 'r') as f:
            data = f.readline().strip().split()
            class_name, x_center, y_center, width, height = map(float, data)

        # Convert YOLO format to OpenCV format
        img_height, img_width, _ = image.shape
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        # Calculate the bounding box coordinates
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # Draw the bounding box on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Display the image with the bounding box
        # cv2.imshow('Image', image)
        cv2.imwrite('/content/drive/MyDrive/Last/visual/' + os.path.splitext(filename)[0] + " copy.jpg", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


# Split dataset

Spliting the dataset into 3 folders(8:1:1) namely :
1. train
2. val
3. test

In [ ]:
import os
import random
import shutil

# Define the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/Last/data2'

# Define the output directories for train, validation, and test sets
train_dir = '/content/drive/MyDrive/Last/self/dataset/train'
val_dir = '/content/drive/MyDrive/Last/self/dataset/val'
test_dir = '/content/drive/MyDrive/Last/self/dataset/test'

# Define the split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Create the output directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get the list of images in the dataset directory
images = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]

# Shuffle the images
random.shuffle(images)

# Calculate the number of images for each set
num_images = len(images)
num_train = int(num_images * train_ratio)
num_val = int(num_images * val_ratio)
num_test = num_images - num_train - num_val

# Split the dataset
train_images = images[:num_train]
val_images = images[num_train:num_train + num_val]
test_images = images[num_train + num_val:]

# Copy the images and annotation files to the respective directories
for image in train_images:
    shutil.copy(os.path.join(dataset_dir, image), os.path.join(train_dir, image))
    shutil.copy(os.path.join(dataset_dir, image.replace('.jpg', '.txt')), os.path.join(train_dir, image.replace('.jpg', '.txt')))

for image in val_images:
    shutil.copy(os.path.join(dataset_dir, image), os.path.join(val_dir, image))
    shutil.copy(os.path.join(dataset_dir, image.replace('.jpg', '.txt')), os.path.join(val_dir, image.replace('.jpg', '.txt')))

for image in test_images:
    shutil.copy(os.path.join(dataset_dir, image), os.path.join(test_dir, image))
    shutil.copy(os.path.join(dataset_dir, image.replace('.jpg', '.txt')), os.path.join(test_dir, image.replace('.jpg', '.txt')))


# Final model run
Training the model(used yolov8s.pt as pretrained model) on different number of epochs and batch size and other hyperparameters

best results on :



In [ ]:
from ultralytics import YOLO
%cd /content/drive/MyDrive/Last/self
model = YOLO('yolov8s.pt')
model.train(data = "/content/drive/MyDrive/Last/self/data.yaml", epochs = 20, batch=8, imgsz=1280, verbose=True, single_cls=True, dropout=0.1, save_conf=True, plots=True)

[Errno 2] No such file or directory: '/content/drive/MyDrive/Last/self'
/content


ERROR: Downloaded file 'yolov8s.pt' does not exist or size is < min_bytes=100000.0
yolov8s.pt missing, try downloading from https://github.com/ultralytics/assets/releases/v0.0.0



KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
%cd /content/drive/MyDrive/final/self_model_tag_removed
model = YOLO('yolov8s.pt')
model.train(data = "/content/drive/MyDrive/final/self_model_tag_removed/data.yaml", epochs = 20, batch=8, imgsz=1280, verbose=True, single_cls=True, dropout=0.1, save_conf=True, plots=True)

# testing

In [ ]:
%cd /content/drive/MyDrive/overfitting/self_learn
model = YOLO("yolov8s.pt")
!yolo task=detect mode=predict model="/content/drive/MyDrive/overfitting/self_learn/runs/detect/train10/weights/best.pt" conf=0.25 source="/content/drive/MyDrive/overfitting/self_learn/testing3" save=true plots=true

2024-04-03 12:01:34.545814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 12:01:34.545959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 12:01:34.550112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 12:01:36.794852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
image 1/10 /content/drive/MyDrive/overfitting/self_learn/testing3/WhatsApp Image 2024-04-03 at 5.

In [ ]:
%cd /content/drive/MyDrive/final/self_model
model = YOLO("yolov8s.pt")
!yolo val model="/content/drive/MyDrive/final/self_model/runs/detect/train2/weights/best.pt" data="/content/drive/MyDrive/final/self_model/data.yaml"

/content/drive/MyDrive/final/self_model
2024-04-20 09:43:35.314469: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 09:43:35.314533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 09:43:35.316409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 09:43:37.261573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/final/self_model/dat

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/final/self_model/runs/detect/train2/weights/best.pt")


# metrics = model.val(data="/content/drive/MyDrive/final/self_model/data.yaml", plots=True, imgsz=640,
#                                batch=16,
#                                conf=0.25,
#                                iou=0.6)

print(model.val(data="/content/drive/MyDrive/final/self_model/data.yaml", plots=True,
                              #  imgsz=640,
                              #  batch=16,
                               conf=0.25,
                               iou=0.6))
# print(metrics)
# metrics.box.map
# metrics.box.map50
# metrics.box.map75
# metrics.box.maps


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/final/self_model/dataset/val.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:22<00:00, 11.26s/it]
                   all         31         31          1      0.935      0.968      0.637
Speed: 5.7ms pre-process, 227.0ms inference, 0.0ms loss, 1.3ms post-process per image


None


In [ ]:
!pip3 install tensorflow==2.13.1

In [ ]:
from ultralytics import YOLO
%cd /content/drive/MyDrive/final
model = YOLO("yolov8s.pt")


model.export(format='tflite')

/content/drive/MyDrive/final


100%|██████████| 21.5M/21.5M [00:00<00:00, 43.0MB/s]


Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 5.1s, saved as 'yolov8s.onnx' (42.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.17.5...
TensorFlow SavedModel: export success ✅ 50.2s, saved as 'yolov8s_saved_model' (107.2 MB)

TensorFlow Lite: starting export with tensorflow 2.13.1...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8s_saved_model/yolov8s_float32.tflite' (42.8 MB)

Export complete (55.1s)
Results saved to /content/drive/MyDrive/final
Predict:         yolo predict task=detect model=yolov8s_saved_model/yolov8s_float32.tflite imgsz=64

'yolov8s_saved_model/yolov8s_float32.tflite'

In [ ]:
from ultralytics import YOLO
%cd /content/drive/MyDrive/final/self_model
# Load a pretrained YOLOv8n model
model = YOLO('/content/drive/MyDrive/final/self_model/runs/detect/train2/weights/best_saved_model/best_float32.tflite')

# Run inference
model.predict('/content/drive/MyDrive/final/self_model/dataset/test/IMG_20240322_144227.jpg', save=True, imgsz=640, conf=0.2)

/content/drive/MyDrive/final/self_model
Loading /content/drive/MyDrive/final/self_model/runs/detect/train2/weights/best_saved_model/best_float32.tflite for TensorFlow Lite inference...

image 1/1 /content/drive/MyDrive/final/self_model/dataset/test/IMG_20240322_144227.jpg: 640x640 2 objects, 556.8ms
Speed: 25.1ms preprocess, 556.8ms inference, 3690.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict4


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'object'}
 obb: None
 orig_img: array([[[ 34,  22,  18],
         [ 38,  26,  22],
         [ 35,  23,  19],
         ...,
         [ 75,  83,  96],
         [ 93,  99, 112],
         [ 95, 101, 114]],
 
        [[ 39,  27,  23],
         [ 39,  27,  23],
         [ 37,  25,  21],
         ...,
         [ 82,  90, 103],
         [ 99, 105, 118],
         [ 99, 105, 118]],
 
        [[ 41,  26,  23],
         [ 42,  27,  24],
         [ 39,  24,  21],
         ...,
         [ 96, 104, 117],
         [111, 119, 132],
         [105, 113, 126]],
 
        ...,
 
        [[169, 169, 155],
         [167, 167, 153],
         [159, 159, 145],
         ...,
         [120, 141, 133],
         [123, 145, 140],
         [124, 146, 141]],
 
        [[162, 163, 147],
         [159, 160, 144],
         [160, 160, 146],
         ...,
         [124, 145